In [0]:
#setting up lakeFS end point access and secret in order to later configure the python client
lakefsEndPoint = 'https://YourEndPoint/' # e.g. 'https://username.azure_region_name.lakefscloud.io'
lakefsAccessKey = 'AKIAlakeFSAccessKey'
lakefsSecretKey = 'lakeFSSecretKey'

In [0]:
# Configuring Python Client
import lakefs
from lakefs.client import Client

clt = Client(
   host=lakefsEndPoint,
   username=lakefsAccessKey,
   password=lakefsSecretKey,
)

In [0]:
# Configuring variables
repositoryName = "learn-lakefs-repo01" # the name of the lakeFS repository
storageNamespace = "https://storage-account-name.blob.core.windows.net/storage-container-name/"  + repositoryName # Unique per repository
sourceBranch = "main"
dataPath = "product-reviews"

In [0]:
# Creating a repository 
repo = lakefs.Repository(
  repositoryName,
  client=clt).create(
    storage_namespace=storageNamespace,
    default_branch=sourceBranch,
    exist_ok=True)
branchMain = repo.branch(sourceBranch)
print(repo)

In [0]:
# Reading data from databricks datasets and inserging into the newly created repository (Creating initial data)
import_data_path = "/databricks-datasets/amazon/test4K/"
df = spark.read.parquet(import_data_path)
df.write.format("parquet").save("lakefs://{}/{}/{}".format(repositoryName,sourceBranch,dataPath))

In [0]:
# Commiting the changes
branchMain.commit(message='Uploading intial data into lakefs', 
        metadata={'using': 'python_api'})

In [0]:
# Review production Data from your production "main" branch

# Note - This example uses static strings instead of parameters for an easier read

df = spark.read.parquet("lakefs://learn-lakefs-repo01/main/product-reviews/")
df.show()

#### More Questions?

###### Join the [lakeFS Slack group](https://lakefs.io/slack)